In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import timeit as timer

from tensorflow.python.keras.layers import Activation, Dense, Conv1D, MaxPooling1D, Dropout, Flatten, LSTM
import warnings
warnings.filterwarnings("ignore")


2022-04-25 15:55:37.085146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 15:55:37.085191: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

xtrain = train.loc[:, train.columns != 'income']
ytrain = train['income']
xtest = test.loc[:, test.columns != 'income']
ytest = test['income']

In [3]:
xtrain['education.num']=np.where(xtrain['education.num'] >= 13, 1, 0) #turned to check if higher than college degree
xtrain['capital.gain']=np.where(xtrain['capital.gain'] > 0, 1, 0)
xtrain['capital.loss']=np.where(xtrain['capital.loss'] > 0, 1, 0)
xtrain['hours.per.week']=np.where(xtrain['hours.per.week'] >= 40, 1, 0) #turned to check if full time

xtrain = xtrain.drop(columns='age')

xtest['education.num']=np.where(xtest['education.num'] >= 13, 1, 0) #turned to check if higher than college degree
xtest['capital.gain']=np.where(xtest['capital.gain'] > 0, 1, 0)
xtest['capital.loss']=np.where(xtest['capital.loss'] > 0, 1, 0)
xtest['hours.per.week']=np.where(xtest['hours.per.week'] >= 40, 1, 0) #turned to check if full time

xtest = xtest.drop(columns='age')

In [4]:
xtrain = xtrain.values.reshape(xtrain.shape[0],xtrain.shape[1],1)
xtrain = tf.cast(xtrain, tf.float32)

xtest = xtest.values.reshape(xtest.shape[0],xtest.shape[1],1)
xtest = tf.cast(xtest, tf.float32)

2022-04-25 15:55:43.774347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-04-25 15:55:43.774532: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-25 15:55:43.774593: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-04-25 15:55:43.774643: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-04-25 15:55:43.774693: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [5]:
ytrain = tf.keras.utils.to_categorical(ytrain, 2)
ytest = tf.keras.utils.to_categorical(ytest, 2)

In [6]:
model = tf.keras.Sequential()


# #two convoluted layers with dropout/pooling
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D())

model.add(Conv1D(filters=120, kernel_size=3, activation='relu'))

model.add(MaxPooling1D())

model.add(Flatten())

model.add(Dense(64, activation='relu'))


#equivalent to 1d sigmoid
model.add(Dense(2, activation='softmax'))
model.compile('adam','categorical_crossentropy', metrics=['accuracy'])
model.build(input_shape= xtrain.shape )

In [7]:
predict = model.predict(np.array( [xtrain[0],] ))

In [8]:
predict[0]

array([0.50468534, 0.49531466], dtype=float32)

In [9]:


t1 = timer.default_timer()
fit_data = model.fit(xtrain,ytrain,epochs=15,batch_size=512, verbose=0)
time_fit = timer.default_timer() - t1
t1 = timer.default_timer()
scores = model.evaluate(xtest,ytest,verbose=0)
time_predict = timer.default_timer() - t1

print("Time fit: ", time_fit)
print("Time predict: ", time_predict)
print("Accuracy: ", scores[1])

Time fit:  21.916256599999542
Time predict:  0.6570763999998235
Accuracy:  0.8325279355049133


In [29]:
xtrain.shape

TensorShape([21727, 43, 1])

In [30]:
xtest.shape

TensorShape([7243, 43, 1])

In [31]:
xtrain[0]

<tf.Tensor: shape=(43, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.]], dtype=float32)>

In [16]:
ytrain[0]

array([1., 0.], dtype=float32)